No seguinte projeto realizaremos uma segmentação de dados baseado na regra RFV em uma base de dados fictícia.

* Bibliotecas:

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

* RFV - Segmentação de dados de clientes baseando-se em recência, frequência e valor.

Os objetivos da segmentação e da clusterização são os mesmos: Criar grupos de dados relevantes que evidênciem um comportamento ou perfil partícular. A diferença é que a clusterização se baseia em algorítmos e a segmentação se baseia em uma regra.

* Dataset:

In [3]:
df = pd.read_csv('data.csv', parse_dates=['DiaCompra'])
df

,ID_cliente,CodigoCompra,DiaCompra,ValorTotal
0,12747,537215,2020-12-05,358.56
1,12747,538537,2020-12-13,347.71
2,12747,541677,2021-01-20,303.04
3,12747,545321,2021-03-01,310.78
4,12747,551992,2021-05-05,442.96
...,...,...,...,...
16122,18283,579673,2021-11-30,223.61
16123,18283,580872,2021-12-06,208.00
16124,18287,554065,2021-05-22,765.28
16125,18287,570715,2021-10-12,1001.32


O dataset contém informações da ID do cliente e da compra, dia da compra e o valor da compra. A partir destes dados criaremos colunas contendo informações sobre a (R)ecência, (F)requência e (V)alor total das compras de cada cliente, a fim de criar grupos que representam o comportamento do cliente.

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16127 entries, 0 to 16126
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   ID_cliente    16127 non-null  int64         
 1   CodigoCompra  16127 non-null  int64         
 2   DiaCompra     16127 non-null  datetime64[ns]
 3   ValorTotal    16127 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2)
memory usage: 504.1 KB


Observando o intervalo de tempo do dataset:

In [5]:
df['DiaCompra'].min()

Timestamp('2020-12-01 00:00:00')

In [6]:
df['DiaCompra'].max()

Timestamp('2021-12-09 00:00:00')

In [7]:
# Definindo como dia atual a data máxima:

dia_atual = df['DiaCompra'].max()
print(dia_atual)

2021-12-09 00:00:00


### Recência

In [8]:
# Distância entre o dia atual e ultima compra do cliente.

df_rec = df.groupby(by='ID_cliente', as_index=False)['DiaCompra'].max()
df_rec.columns = ['ID_cliente', 'DiaUltimaCompra']
df_rec

,ID_cliente,DiaUltimaCompra
0,12747,2021-12-07
1,12748,2021-12-09
2,12749,2021-12-06
3,12820,2021-12-06
4,12821,2021-05-09
...,...,...
3897,18280,2021-03-07
3898,18281,2021-06-12
3899,18282,2021-12-02
3900,18283,2021-12-06


In [9]:
df_rec['recencia'] = df_rec['DiaUltimaCompra'].apply(
    lambda x: (dia_atual - x).days
)

In [10]:
df_rec.sort_values('recencia', ascending=False)

,ID_cliente,DiaUltimaCompra,recencia
3703,18011,2020-12-01,373
3629,17908,2020-12-01,373
1351,14729,2020-12-01,373
2298,16048,2020-12-01,373
3745,18074,2020-12-01,373
...,...,...,...
433,13426,2021-12-09,0
3257,17389,2021-12-09,0
3236,17364,2021-12-09,0
2989,17001,2021-12-09,0


In [11]:
df_rec.drop('DiaUltimaCompra', axis=1, inplace=True)
df_rec

,ID_cliente,recencia
0,12747,2
1,12748,0
2,12749,3
3,12820,3
4,12821,214
...,...,...
3897,18280,277
3898,18281,180
3899,18282,7
3900,18283,3


### Frequência

In [12]:
df_freq = df.groupby('ID_cliente')['CodigoCompra'].count().reset_index()
df_freq.columns = ['ID_cliente', 'frequencia']
df_freq

,ID_cliente,frequencia
0,12747,11
1,12748,178
2,12749,5
3,12820,4
4,12821,1
...,...,...
3897,18280,1
3898,18281,1
3899,18282,2
3900,18283,14


### Valor

In [13]:
df_val = df.groupby('ID_cliente')['ValorTotal'].sum().reset_index()
df_val.columns = ['ID_cliente', 'valor']
df_val

,ID_cliente,valor
0,12747,4196.01
1,12748,31533.04
2,12749,4090.88
3,12820,942.34
4,12821,92.72
...,...,...
3897,18280,180.60
3898,18281,80.82
3899,18282,178.05
3900,18283,2090.43


Tabela RFV:

In [14]:
df_rfv = df_rec.merge(df_freq, on='ID_cliente')
df_rfv = df_rfv.merge(df_val, on ='ID_cliente')
df_rfv.set_index('ID_cliente', inplace=True)

df_rfv

,recencia,frequencia,valor
ID_cliente,,,
12747,2,11,4196.01
12748,0,178,31533.04
12749,3,5,4090.88
12820,3,4,942.34
12821,214,1,92.72
...,...,...,...
18280,277,1,180.60
18281,180,1,80.82
18282,7,2,178.05


### Segmentação

A segmentação será criada a partir dos quartis das variáveis RFV:

In [15]:
quartis = df_rfv.quantile(q=[0.25, 0.5, 0.75])
quartis

,recencia,frequencia,valor
0.25,17.0,1.0,299.705
0.50,50.0,2.0,643.555
0.75,143.0,5.0,1533.600


In [16]:
quartis.to_dict()

{'recencia': {0.25: 17.0, 0.5: 50.0, 0.75: 143.0},
 'frequencia': {0.25: 1.0, 0.5: 2.0, 0.75: 5.0},
 'valor': {0.25: 299.70500000000004, 0.5: 643.555, 0.75: 1533.6}}

In [17]:
# Definindo funções que atribuem um 'score' para valores nos quartis:

def rec_class(x, r, q_dict):
    """Classifica como melhor o menor quartil
    x = valor da linha,
    r = recencia
    q_dict = dicionario do quartil"""

    if x <= q_dict[r][0.25]:
        return 'A'
    elif x <= q_dict[r][0.5]:
        return 'B'
    elif x <= q_dict[r][0.75]:
        return 'C'
    else:
        return 'D'
    
def freq_val_class(x, fv, q_dict):
    """Classifica como melhor o menor quartil
    x = valor da linha,
    fv = frequencia ou valor
    q_dict = dicionario do quartil"""

    if x <= q_dict[fv][0.25]:
        return 'D'
    elif x <= q_dict[fv][0.5]:
        return 'C'
    elif x <= q_dict[fv][0.75]:
        return 'B'
    else:
        return 'A'   

In [18]:
# Criando as colunas com os scores de cada variável:

df_rfv['r_quartil'] = df_rfv['recencia'].apply(rec_class, args=('recencia', quartis))

df_rfv['f_quartil'] = df_rfv['frequencia'].apply(freq_val_class, args=('frequencia', quartis))

df_rfv['v_quartil'] = df_rfv['valor'].apply(freq_val_class, args=('valor', quartis))

In [19]:
df_rfv.head()

,recencia,frequencia,valor,r_quartil,f_quartil,v_quartil
ID_cliente,,,,,,
12747,2,11,4196.01,A,A,A
12748,0,178,31533.04,A,A,A
12749,3,5,4090.88,A,B,A
12820,3,4,942.34,A,B,B
12821,214,1,92.72,D,D,D


In [20]:
# Criando uma coluna com o score RFV:

df_rfv['score'] = (df_rfv['r_quartil'] + df_rfv['f_quartil'] + df_rfv['v_quartil'])

df_rfv.head()

,recencia,frequencia,valor,r_quartil,f_quartil,v_quartil,score
ID_cliente,,,,,,,
12747,2,11,4196.01,A,A,A,AAA
12748,0,178,31533.04,A,A,A,AAA
12749,3,5,4090.88,A,B,A,ABA
12820,3,4,942.34,A,B,B,ABB
12821,214,1,92.72,D,D,D,DDD


In [21]:
# Contagem de clientes em cada grupo:

df_rfv['score'].value_counts()

score
AAA    417
DDD    402
DDC    212
BBB    187
CDD    186
BAA    166
ABB    153
CDC    140
BDD    139
CBB    133
CBA     96
ABA     87
DCC     87
BDC     85
CCB     83
BBA     81
BCC     79
CCC     79
ACC     65
BCB     65
CAA     63
CBC     60
DCB     57
DCD     54
ADD     53
BBC     53
AAB     47
ACB     46
CDB     43
BCD     43
ABC     42
DBB     41
DBC     37
DDB     36
CCD     35
BAB     35
ADC     32
ACD     32
CAB     23
BDB     20
DAA     14
DBA     13
BCA     11
CCA     10
CBD     10
DBD      9
ABD      7
DCA      6
CDA      6
BBD      5
DDA      3
ADB      3
DAB      3
AAC      3
ACA      2
BDA      1
CAC      1
AAD      1
Name: count, dtype: int64

Uma vez classificados em grupos, podemos tomar ações customizadas para cada grupo:

In [22]:
dict_acoes = {
    'AAA':
    'Enviar cupons de desconto, Pedir para indicar nosso produto pra algum amigo, Ao lançar um novo produto enviar amostras grátis pra esses.',
    'DDD':
    'Churn! clientes que gastaram bem pouco e fizeram poucas compras, fazer nada',
    'DAA':
    'Churn! clientes que gastaram bastante e fizeram muitas compras, enviar cupons de desconto para tentar recuperar',
    'CAA':
    'Churn! clientes que gastaram bastante e fizeram muitas compras, enviar cupons de desconto para tentar recuperar'
}

In [23]:
df_rfv['acoes_de_marketing/crm'] = df_rfv['score'].map(dict_acoes)

In [24]:
df_rfv.head()

,recencia,frequencia,valor,r_quartil,f_quartil,v_quartil,score,acoes_de_marketing/crm
ID_cliente,,,,,,,,
12747,2,11,4196.01,A,A,A,AAA,"Enviar cupons de desconto, Pedir para indicar ..."
12748,0,178,31533.04,A,A,A,AAA,"Enviar cupons de desconto, Pedir para indicar ..."
12749,3,5,4090.88,A,B,A,ABA,NaN
12820,3,4,942.34,A,B,B,ABB,NaN
12821,214,1,92.72,D,D,D,DDD,Churn! clientes que gastaram bem pouco e fizer...
